In [256]:
import requests
import requests_cache
from matplotlib import pyplot as plt
import lxml.html
import pandas as pd
import re
import json 
import lxml.html as lx
import urllib
from urllib2 import Request, urlopen, HTTPError
from urlparse import urlunparse, urlparse, urljoin
import numpy

In [272]:
def fetch_page(url):
    response = requests.get(url)
    response.raise_for_status()
    return response

url="https://www.zillow.com/browse/homes/ca/"
requests_cache.install_cache("cache")
page = fetch_page(url)
html = lx.fromstring(page.text)
county_html = html.xpath('//main/div/div/ul/li/a')
county = [i.text_content() for i in county_html]
county_name = pd.DataFrame({"RegionName":[" ".join(k) for k in [i.split(" ")[:-2] for i in county]]})

In [455]:
#Import Housing Data
import os.path


def dataframe(i = 0):
    
    """Read datasets
    Please change the file path to fit to your computer
    Input:the name of dataset //dataname   
    """
    
    file_path = "/Users/HUI/Dropbox/16_UCDavis/17W_STA141B/05_Project/Housing_Data"
    file_name = ["County_MedianSoldPrice_AllHomes.csv","County_MedianListingPrice_AllHomes.csv","County_MedianListingPrice_1Bedroom.csv",
                 "County_MedianListingPrice_2Bedroom.csv","County_MedianListingPrice_3Bedroom.csv","County_MedianListingPrice_4Bedroom.csv",
                 "County_MedianListingPrice_5BedroomOrMore.csv","County_MedianListingPrice_CondoCoop.csv","County_MedianListingPrice_DuplexTriplex.csv",
                 "County_MedianListingPrice_Sfr.csv"]
    dataframe = pd.read_csv(os.path.join(file_path,file_name[i]),header=0)
    CA_dataframe = dataframe[dataframe["State"]== "CA"]
    CA_dataframe = CA_dataframe.reset_index(drop=True)
    return(CA_dataframe)

###name of eachdata
dataname=["sold","list","one","two","thre","four","five","cond","trip","sfr"]

###save all of dataset to CA dict.
CA = {i: dataframe(dataname.index(i)) for i in dataname}

In [456]:
#All Home Sold time-series data 1996-2016
CA["sold"] = pd.merge(pd.DataFrame(county_name),CA["sold"], how='left', on=['RegionName'])
CA_sold_t = CA["sold"].transpose()
CA_sold_t = CA_sold_t.rename(columns=lambda i: CA["sold"]["RegionName"][i])[7:]
date = pd.date_range('1996-04', '2016-07', freq='M')
CA_sold_t['date'] = date

#sold price time-series data
CA_sold_t = CA_sold_t.set_index('date')
CA_sold_t.head()

,Los Angeles,Orange,San Diego,Riverside,San Bernardino,Santa Clara,Alameda,Sacramento,Contra Costa,Fresno,...,Glenn,Plumas,Colusa,Mariposa,Inyo,Trinity,Mono,Modoc,Sierra,Alpine
date,,,,,,,,,,,,,,,,,,,,,
1996-04-30,162225,191525,158225,114550,100250,234325,196275,120425,192750,94850,...,67000,NaN,NaN,NaN,121471,53000,77500,NaN,NaN,NaN
1996-05-31,162925,197550,158125,112500,101700,238575,196900,121675,203000,95475,...,65823.5,NaN,97230.8,115000,NaN,NaN,78970.6,NaN,NaN,NaN
1996-06-30,164500,197875,158925,115000,103450,238875,202225,122175,209050,95475,...,NaN,NaN,NaN,NaN,119077,51950,NaN,NaN,NaN,NaN
1996-07-31,164550,198050,158700,113975,101850,241875,203700,122175,207150,96550,...,NaN,NaN,NaN,94230.8,112882,NaN,81538.5,NaN,NaN,NaN
1996-08-31,163300,199300,157300,113225,101650,241200,202250,121075,202500,96625,...,NaN,80000,NaN,NaN,NaN,82300,98470.6,NaN,NaN,NaN


In [457]:
#Different type of house average yr 2015/2016
def aver(i="one"):
    data = CA[i]
    CA_2015 = data.iloc[:,[k.__contains__("2015") for k in data.keys()]]
    CA_2016 = data.iloc[:,[k.__contains__("2016") for k in data.keys()]]
    CA_mean = pd.DataFrame({i+"_15":CA_2015.mean(1),i+"_16":CA_2016.mean(1)})
    CA_ave_name = pd.DataFrame({"RegionName":data["RegionName"]})
    CA_ave = pd.concat([CA_ave_name,CA_mean],axis=1)
    return CA_ave
CA_ave=[aver(i) for i in dataname]


#average price in yr 25/26 for different type of house/and all homes
diff_ave = reduce(lambda left,right: pd.merge(left,right,how="left",on='RegionName'), CA_ave)
diff_ave.head()

,RegionName,sold_15,sold_16,list_15,list_16,one_15,one_16,two_15,two_16,thre_15,...,four_15,four_16,five_15,five_16,cond_15,cond_16,trip_15,trip_16,sfr_15,sfr_16
0,Los Angeles,498737.500000,518508.333333,537666.666667,587328.666667,363468.333333,409424.500000,433116.583333,469545.750000,497107.916667,...,659808.333333,701394.000000,1.038750e+06,1.176233e+06,452570.833333,488478.750000,664705.500000,7.312750e+05,576524.500000,629662.333333
1,Orange,601020.833333,623150.000000,675932.083333,705890.000000,279250.000000,303945.833333,417687.500000,441220.833333,639658.333333,...,841700.000000,883811.583333,1.382648e+06,1.439552e+06,422483.333333,446204.166667,924108.583333,1.120979e+06,787609.750000,820343.541667
2,San Diego,472187.500000,489250.000000,538650.000000,575525.000000,276475.000000,319328.625000,360778.166667,396704.083333,494828.166667,...,684390.666667,716899.958333,1.093432e+06,1.184135e+06,376438.583333,410828.333333,678408.333333,7.657750e+05,609515.250000,641470.583333
3,Riverside,309068.750000,320437.500000,344257.500000,361732.458333,145399.875000,145940.750000,227945.750000,237962.041667,327233.333333,...,394170.750000,413642.833333,4.613150e+05,4.925966e+05,273656.666667,280891.666667,352866.583333,3.872125e+05,356255.250000,376051.708333
4,San Bernardino,264327.083333,272666.666667,271312.500000,291733.333333,NaN,NaN,172779.083333,185468.166667,255624.833333,...,367520.833333,385991.166667,4.659978e+05,4.931395e+05,259899.916667,271735.416667,295638.125000,3.068167e+05,272363.916667,293408.333333
